# Ensemble Pytorch Models With H-Flip TTA

## Load libraries and define model parameters

In [1]:
import os, gc, pickle
import cv2
import pdb
import time

import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt

from albumentations import (HorizontalFlip, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)
from albumentations.torch import ToTensor

import segmentation_models_pytorch as smp

In [2]:
def run_length_decode(rle, height=1024, width=1024, fill_value=1):
    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)
    start = 0
    for index, length in rle:
        start = start+index
        end = start+length
        component[start: end] = fill_value
        start = end
    component = component.reshape(width, height).T
    return component

def run_length_encode(component):
    component = component.T.flatten()
    start = np.where(component[1:] > component[:-1])[0]+1
    end = np.where(component[:-1] > component[1:])[0]+1
    length = end-start
    rle = []
    for i in range(len(length)):
        if i == 0:
            rle.extend([start[0], length[0]])
        else:
            rle.extend([start[i]-end[i-1], length[i]])
    rle = ' '.join([str(r) for r in rle])
    return rle

def post_process(probability, threshold, min_size):
    mask = cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1]
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((1024, 1024), np.float32)
    num = 0
    for c in range(1, num_component):
        p = (component == c)
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

In [3]:
WORK_DIR='../output/20190903_s2'
INPUT_TEST_DIR_1024 = "../input/1024_s2/test"

sample_submission_file='../output/20190829_s2/submission_swa_512_5ave_0p55th_minpix3500_s2_keras.csv' #to match keras model order
submission_file='/home/akuritsyn/projects/pneumothorax/output/20190903_s2/submission_pytorch_5fold_ave_Wflip_0p55th.csv'

In [4]:
PYTORCH_MODELS_DIR='../output/20190903_pytorch_results_s2'

path_models=[PYTORCH_MODELS_DIR+'/1024_stage2_model_fold0.pth',\
             PYTORCH_MODELS_DIR+'/1024_stage2_model_fold1.pth',\
             PYTORCH_MODELS_DIR+'/1024_stage2_model_fold2.pth',\
             PYTORCH_MODELS_DIR+'/1024_stage2_model_fold3.pth',\
             PYTORCH_MODELS_DIR+'/1024_stage2_model_fold4.pth']

In [5]:
#Model parameters

size = 1024
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
num_workers = 4 #8
batch_size = 1 #4
best_threshold = 0.55
min_size = 3500
device = torch.device("cuda:0")
df = pd.read_csv(sample_submission_file)

## Perform inference on test images

In [6]:
class TestDataset(Dataset):
    def __init__(self, root, df, size, mean, std, tta=4):
        self.root = root
        self.size = size
        self.fnames = list(df["ImageId"])
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Normalize(mean=mean, std=std, p=1), #p=1
                Resize(size, size),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname + ".png")
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return images

    def __len__(self):
        return self.num_samples   

In [7]:
testset = DataLoader(
    TestDataset(INPUT_TEST_DIR_1024, df, size, mean, std),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

model_0 = smp.Unet("resnet34", encoder_weights=None, activation=None)
model_0.eval()

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [7]:
probs=[]

for i, batch in enumerate(tqdm(testset)):
     
    for j in range(0,5): #average over 5 folds

        checkpoint0= torch.load(path_models[j], map_location=lambda storage, loc: storage)
        model_0.load_state_dict(checkpoint0["state_dict"])
        model_0.to(device)
        
        if j==0:
            preds = torch.sigmoid(model_0(batch.to(device)))
        else:
            preds = preds+torch.sigmoid(model_0(batch.to(device)))
         
        model_0.cpu()
    
    preds=preds/5.
    preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    for probability in preds:
        if probability.shape != (1024, 1024):
            probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)
        probs.append(probability)      

with open(WORK_DIR+'/probs_1024.pkl', 'wb') as f:
     pickle.dump(probs, f)

In [ ]:
# del probs
# gc.collect()

In [ ]:
# with open(WORK_DIR+'/probs_1024.pkl', 'rb') as f:
#     probs=pickle.load(f)

## Perform inference on h-flipped test images (may need to restart kernel to free up memory before proceeding)

In [7]:
class TestDataset_Flipped(Dataset):
    def __init__(self, root, df, size, mean, std):
        self.root = root
        self.size = size
        self.fnames = list(df["ImageId"])
        self.num_samples = len(self.fnames)
        self.transform = Compose(
            [
                Normalize(mean=mean, std=std, p=1),
                Resize(size, size),
                HorizontalFlip(True,p=1),
                ToTensor(),
            ]
        )

    def __getitem__(self, idx):
        fname = self.fnames[idx]
        path = os.path.join(self.root, fname + ".png")
        image = cv2.imread(path)
        images = self.transform(image=image)["image"]
        return images

    def __len__(self):
        return self.num_samples


#flipped_prob
testset_flipped = DataLoader(
    TestDataset_Flipped(INPUT_TEST_DIR_1024, df, size, mean, std),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

probs_flipped=[]
for i, batch in enumerate(tqdm(testset_flipped)):
    
    for j in range(0,5): 
        checkpoint0= torch.load(path_models[j], map_location=lambda storage, loc: storage)
        model_0.load_state_dict(checkpoint0["state_dict"])
        model_0.to(device)
        if j==0:
            preds = torch.sigmoid(model_0(batch.to(device)))
        else:
            preds = preds+torch.sigmoid(model_0(batch.to(device)))
        model_0.cpu()
        #del model_0
        #gc.collect()
        
    preds=preds/5.
    preds = preds.detach().cpu().numpy()[:, 0, :, :] # (batch_size, 1, size, size) -> (batch_size, size, size)
    for probability in preds:
        if probability.shape != (1024, 1024):
            probability = cv2.resize(probability, dsize=(1024, 1024), interpolation=cv2.INTER_LINEAR)
        probs_flipped.append(np.fliplr(probability))

with open(WORK_DIR+'/probs_flipped_1024.pkl', 'wb') as f:
     pickle.dump(probs_flipped, f)  

In [8]:
del model_0
gc.collect()

27

## Average predicitions for normal and flipped images and generate submission file

In [ ]:
# with open(WORK_DIR+'/probs_flipped_1024.pkl', 'rb') as f:
#     probs_flipped=pickle.load(f)

with open(WORK_DIR+'/probs_1024.pkl', 'rb') as f:
    probs=pickle.load(f)

In [10]:
for i in range(len(probs)):
    probs[i]=(probs[i]+probs_flipped[i])/2.
    
del probs_flipped
gc.collect()

# with open(WORK_DIR+'/probs_ave_1024.pkl', 'wb') as f:
#      pickle.dump(probs, f) 

In [ ]:
del probs
gc.collect()

In [ ]:
# with open(WORK_DIR+'/probs_ave_1024.pkl', 'rb') as f:
#     probs=pickle.load(f) 

In [13]:
# Generate submission file

encoded_pixels = []

for probability in probs:
    predict, num_predict = post_process(probability, best_threshold, min_size)
    if num_predict == 0:
        encoded_pixels.append('-1')
    else: 
        r = run_length_encode(predict)
        encoded_pixels.append(r)
df['EncodedPixels'] = encoded_pixels
df.to_csv(submission_file, columns=['ImageId', 'EncodedPixels'], index=False)

In [15]:
df.head(10)

,ImageId,EncodedPixels
0,ID_743b284f8,-1
1,ID_ec235550f,148308 37 979 60 958 77 939 94 921 111 905 131...
2,ID_76747ee35,170718 8 999 32 982 43 971 54 962 61 958 65 95...
3,ID_71d2042e6,248138 6 1014 12 1010 15 1007 18 1004 20 1002 ...
4,ID_6023cb909,-1
5,ID_b439e7bb3,-1
6,ID_93ace8dc6,-1
7,ID_bf9328240,-1
8,ID_f9fa52a40,-1
9,ID_8a837104a,588929 5 45 6 965 14 35 12 962 20 27 16 959 34...
